# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [1]:
import subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
import re

A continuación leemos los datasets en dataframes de Pandas.

In [2]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

df_tsplib = pd.read_csv("experimentos/tsplib.csv");
df_nuestros = pd.read_csv("experimentos/nuestros.csv");
df_mismotamanio = pd.read_csv("experimentos/mismo_tamanio.csv");

La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.
- H-CG: Heuristica constructiva golosa.
- H-AGM: Heuristica con AGM.
- BL: Busqueda local.
- BL-Tabu: Tabu search

In [3]:
def correr_experimento(*args):
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../main", *args], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion para el algoritmo: {args}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    [tamanio, costo_resultado] = process.stdout.read().split('\n')[0].split(" ");

    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return [tiempo_de_ejecucion, tamanio, costo_resultado];

## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [4]:
experimentos = [];

## Heuristica Constructiva Golosa
Agregar Heurisitica Golosa sobre todas las instancias de tsplib.

In [ ]:
for index, row in df_tsplib.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "H-CG"]);

Agregar Heurisitica Golosa sobre todas las instancias aleatorias.

In [5]:
for index, row in df_nuestros.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "H-CG"]);

Agregar Heurisitica Golosa sobre todas las instancias de tamaño 50.

In [ ]:
for index, row in df_mismotamanio.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "H-CG"]);

## Heurisitica AGM
Correr Heurisitica AGM sobre todas las instancias de tsplib.

In [ ]:
for index, row in df_tsplib.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "H-AGM"]);

Agregar Heurisitica AGM sobre todas las instancias aleatorias.

In [6]:
for index, row in df_nuestros.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "H-AGM"]);

Agregar Heurisitica AGM sobre todas las instancias de tamaño 50.

In [ ]:
for index, row in df_mismotamanio.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "H-AGM"]);

## Busqueda Local
Correr Busqueda Local sobre todas las instancias de tsplib.

In [ ]:
for index, row in df_tsplib.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "BL"]);

Agregar Busqueda Local sobre todas las instancias aleatorias.

In [7]:
for index, row in df_nuestros.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "BL"]);

Agregar Busqueda Local sobre todas las instancias de tamaño 50.

In [ ]:
for index, row in df_mismotamanio.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "BL"]);

## Tabu Search
Correr Tabu Search sobre todas las instancias de tsplib.

In [ ]:
for tipo_memoria in ['ciclos']:
    for T in [10, 100, 1000]:
        for iteraciones in [100, 500, 1000]:
            for index, row in df_tsplib.iterrows():
                if (int(re.findall(r'\d+', row["instancia"])[0]) > 100): continue
                experimentos.append([row["instancia"], row["ruta_instancia"], "BL-Tabu", tipo_memoria, str(iteraciones), str(T)])

Agregar Tabu Search sobre todas las instancias aleatorias.

In [8]:
for index, row in df_nuestros.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "BL-Tabu", "ciclos", str(1000), str(1000)])

Agregar Tabu Search sobre todas las instancias de tamaño 50.

In [ ]:
for index, row in df_mismotamanio.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "BL-Tabu", "ciclos", str(1000), str(1000)]);

## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

In [9]:
columnas = ["instancia", "metodo", "tiempo", 'tamaño', 'costo', 'tipo_memoria', 'iteraciones', 'T'];
filas = [];
numero = 1
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    tamanio_instancia = 0
    costo_resultado = 0
    T = 5
    for i in range(0, T):
        [tiempo, tamanio, costo] = correr_experimento(experimento[2], experimento[1], *experimento[3:])
        tiempos.append(tiempo);
        tamanio_instancia = tamanio
        costo_resultado = costo
    tiempo = np.median(tiempos);
    if len(experimento) == 3:
        experimento.append(None)
        experimento.append(None)
        experimento.append(None)
    filas.append([experimento[0], experimento[2], tiempo, tamanio, costo, *experimento[3:]]);
    print([experimento[0], experimento[2], tiempo, tamanio, costo, *experimento[3:]])
    df_resultado = pd.DataFrame(filas, columns=columnas);
    df_resultado.to_csv("resultados/resultado-aleatorias.csv", index=False, header=True);


'Experimento: 360/360'

[99, 'BL-Tabu', 13406.9, '99', '473', 'ciclos', '1000', '1000']
